1.
Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(name)
FROM company
WHERE status LIKE '%closed%'

2.
Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
SELECT funding_total
FROM company
WHERE category_code LIKE '%news%'
AND country_code = 'USA'
ORDER BY funding_total DESC

3.
Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
AND acquired_at BETWEEN '01.01.2011' AND '31.12.2013'

4.
Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
last_name,
twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'

5.
Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%'

6.
Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC

7.
Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at,
MIN(raised_amount),
MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0
AND MIN(raised_amount) <> MAX(raised_amount)

8.
Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
CASE 
WHEN invested_companies >= 100 THEN 'high_activity'
WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
WHEN invested_companies < 20 THEN 'low_activity'
END
FROM fund

9.
Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds)) ASC

10.
Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
MIN(invested_companies),
MAX(invested_companies),
AVG(invested_companies)
FROM fund
WHERE EXTRACT (YEAR FROM CAST(founded_at AS timestamp)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) <> 0
ORDER BY AVG(invested_companies) DESC , country_code

LIMIT 10

11.
Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT 
 p.first_name,
 p.last_name,
 e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id=e.person_id

12.
Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT c.name,
       COUNT(DISTINCT e.instituition) AS quantity
FROM company AS c
LEFT JOIN people AS p ON p.company_id = c.id
JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY quantity DESC 
LIMIT 5;

13.
Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT c.name
FROM company AS c INNER JOIN funding_round AS f ON c.id = f.company_id
WHERE c.status = 'closed'
AND f.is_first_round = 1
AND f.is_last_round = 1

14.
Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT id
FROM people 
WHERE company_id IN (SELECT id
                     FROM company
                     WHERE status = 'closed'
                     AND id IN (SELECT company_id
                                FROM funding_round
                                WHERE is_first_round = 1
                                AND is_last_round = 1))
             


15.
Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT person_id,
instituition
FROM education
WHERE person_id IN(
SELECT id
FROM people 
WHERE company_id IN (SELECT id
                     FROM company
                     WHERE status = 'closed'
                     AND id IN (SELECT company_id
                                FROM funding_round
                                WHERE is_first_round = 1
                                AND is_last_round = 1)))

16.
Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT p.id,
COUNT(e.instituition) AS counts
FROM people AS p
JOIN company AS c ON p.company_id = c.id
JOIN education AS e ON p.id=e.person_id
WHERE status = 'closed'
AND c.id IN ( SELECT company_id 
             FROM funding_round
             WHERE is_last_round = 1 AND is_first_round = 1)
GROUP BY p.id

17.
Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(b.counts)

FROM
(SELECT p.id,
COUNT(e.instituition) AS counts
FROM people AS p
JOIN company AS c ON p.company_id = c.id
JOIN education AS e ON p.id=e.person_id
WHERE status = 'closed'
AND c.id IN ( SELECT company_id 
             FROM funding_round
             WHERE is_last_round = 1 AND is_first_round = 1)
GROUP BY p.id) AS b

18.
Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
WITH

i AS (SELECT COUNT(instituition) AS avg_instituition , person_id
      FROM education
      GROUP BY person_id),
      
w AS (SELECT company_id, id
      FROM people),
      
q AS (SELECT name, id
      FROM company
      WHERE name = 'Facebook')
      
SELECT AVG(i.avg_instituition)  
FROM i JOIN w ON i.person_id = w.id
JOIN q ON w.company_id = q.id

19.
Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:

SELECT g.name_of_fund name_of_fund,
       g.name_of_company name_of_company,
       g.amount amount
FROM        
        (SELECT i.fund_id,
               f.name name_of_fund,
               f.milestones milestones,
               i.company_id,
               c.name name_of_company,
               i.funding_round_id,
               fr.raised_amount amount
        FROM investment i
        JOIN fund f ON f.id = i.fund_id
        JOIN funding_round fr ON fr.id = i.funding_round_id
        JOIN company c ON c.id = i.company_id
        WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) IN (2012, 2013) AND c.milestones > 6) AS g

20.
Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT c1.name as comp_buyer,
a.price_amount,
c2.name as comp_sell,
c2.funding_total,
ROUND(a.price_amount/c2.funding_total)
FROM acquisition AS a
JOIN company AS c1 ON a.acquiring_company_id=c1.id
JOIN company AS c2 ON a.acquired_company_id=c2.id
WHERE a.price_amount > 0 AND c2.funding_total > 0
ORDER BY a.price_amount DESC
LIMIT 10

21.
Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name,
EXTRACT (MONTH FROM CAST(fr.funded_at AS date)) AS month
FROM company AS c
LEFT JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.category_code LIKE '%social%'
AND EXTRACT (YEAR FROM CAST(fr.funded_at AS date)) BETWEEN '2010' AND '2013'
AND fr.raised_amount  > 0

22.
Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
fund_quantity AS (SELECT EXTRACT(MONTH FROM CAST(funded_at AS date)) AS month,
       COUNT(DISTINCT f.name) AS total_funds
       FROM funding_round AS fr
       JOIN investment AS i ON i.funding_round_id = fr.id
       JOIN fund AS f ON f.id = i.fund_id
       WHERE (EXTRACT(YEAR FROM CAST(funded_at AS date)) >= 2010
         AND EXTRACT(YEAR FROM CAST(funded_at AS date)) <= 2013)
         AND country_code = 'USA'
       GROUP BY month),
acquired_companies AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS date)) AS month,
       COUNT(acquired_company_id) AS total_companies,
       SUM(price_amount) AS total_amount
       FROM acquisition
       WHERE (EXTRACT(YEAR FROM CAST(acquired_at AS date)) >= 2010
         AND EXTRACT(YEAR FROM CAST(acquired_at AS date)) <= 2013)                
       GROUP BY month)
SELECT fund_quantity.month,
       total_funds,
       total_companies,
       total_amount
FROM fund_quantity LEFT JOIN acquired_companies ON fund_quantity.month = acquired_companies.month
ORDER BY fund_quantity.month

23.
Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH

    y2011 AS (SELECT country_code AS country,
                 AVG(funding_total) AS average_2011
                 FROM company
                 WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
                 GROUP BY country),
                 
    y2012 AS (SELECT country_code AS country,
                 AVG(funding_total) AS average_2012
                 FROM company
                 WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
                 GROUP BY country),
                 
    y2013 AS (SELECT country_code AS country,
                 AVG(funding_total) AS average_2013
                 FROM company
                 WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
                 GROUP BY country)
    
SELECT y2011.country,
       y2011.average_2011,
       y2012.average_2012,
       y2013.average_2013
FROM  y2011
JOIN y2012 ON y2011.country = y2012.country
JOIN y2013 ON y2011.country = y2013.country
ORDER BY y2011.average_2011 DESC ;